# Visualize adversarial training metrics

### Read and parse logs

In [1]:
from pathlib import Path
import os
import json
import pandas as pd

LOGS_PATH = Path('/cluster/scratch/thobauma/data/models')

In [2]:
LOGS_PATH

PosixPath('/cluster/scratch/thobauma/data/models')

In [3]:
def read_log_file(path):
    # reading the data from the file
    with open(path) as f:
        data = f.read()
        
    content = []
    
    for line in data.split('\n'):
        if len(line):
            content.append(json.loads(line))
    
    return content

In [4]:
logs = {}

In [5]:
for p in os.listdir(LOGS_PATH):
    
    if os.path.isdir(Path(LOGS_PATH, p)):
        logs[p] = read_log_file(Path(LOGS_PATH, p, 'log.txt'))
        

In [6]:
logs.keys()

dict_keys(['pgd_1', 'pgd_05', 'pgd_001', 'pgd_01', 'pgd_003', 'pgd_03', 'pgd_007'])

In [12]:
logs

{'pgd_1': [{'train_lr': 0.0002499999999999914,
   'train_loss': 2.266011511554314,
   'epoch': 0,
   'test_loss': 2.175616983781781,
   'test_acc1': 40.05555555555556,
   'test_acc5': 88.13888888888889,
   'test_adv_loss': 2.213286981248019,
   'test_adv_acc1': 1.9722222222222223,
   'test_adv_acc5': 20.805555555555557},
  {'train_lr': 0.00022612712429687071,
   'train_loss': 2.204873303232825,
   'epoch': 1,
   'test_loss': 2.1906050004457174,
   'test_acc1': 21.77777777777778,
   'test_acc5': 70.0,
   'test_adv_loss': 2.2012952796199867,
   'test_adv_acc1': 7.444444444444445,
   'test_adv_acc5': 43.19444444444444},
  {'train_lr': 0.00016362712429687207,
   'train_loss': 2.200054692961301,
   'epoch': 2,
   'test_loss': 2.192425204996477,
   'test_acc1': 21.694444444444443,
   'test_acc5': 67.02777777777777,
   'test_adv_loss': 2.1998245046849836,
   'test_adv_acc1': 9.527777777777779,
   'test_adv_acc5': 47.666666666666664},
  {'train_lr': 8.637287570313334e-05,
   'train_loss': 2.19

In [8]:
# Prepare data
x_values = []
clean_acc1_values = []
adv_acc1_values = []
test_adv_loss_values = []
test_clean_loss_values = []

for l in logs:
    x_values.append(float('0.'+l.replace('pgd_', '')))

    clean_acc1_values.append(logs[l][2]['test_acc1'])
    adv_acc1_values.append(logs[l][2]['test_adv_acc1'])
    
    test_adv_loss_values.append(logs[l][2]['test_adv_loss'])
    test_clean_loss_values.append(logs[l][2]['test_loss'])

In [9]:
df = pd.DataFrame({'x': x_values, 
                   'clean_acc1_values': clean_acc1_values, 
                   'adv_acc1_values': adv_acc1_values,
                   'test_clean_loss_values': test_clean_loss_values, 
                   'test_adv_loss_values': test_adv_loss_values})

In [13]:
df

,x,clean_acc1_values,adv_acc1_values,test_clean_loss_values,test_adv_loss_values
0,0.100,21.694444,9.527778,2.192425,2.199825
1,0.050,19.250000,9.666667,2.195052,2.199560
2,0.001,97.361111,91.694444,0.095505,0.257809
3,0.010,73.444444,16.722222,2.085213,2.187268
4,0.003,96.138889,69.583333,0.245459,0.895816
5,0.030,16.000000,7.111111,2.194720,2.199584
6,0.007,77.611111,29.472222,1.554361,2.029128


In [14]:
df = df.sort_values(by=['x'])
df

,x,clean_acc1_values,adv_acc1_values,test_clean_loss_values,test_adv_loss_values
2,0.001,97.361111,91.694444,0.095505,0.257809
4,0.003,96.138889,69.583333,0.245459,0.895816
6,0.007,77.611111,29.472222,1.554361,2.029128
3,0.010,73.444444,16.722222,2.085213,2.187268
5,0.030,16.000000,7.111111,2.194720,2.199584
1,0.050,19.250000,9.666667,2.195052,2.199560
0,0.100,21.694444,9.527778,2.192425,2.199825


### Plotting

In [15]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Legend
from bokeh.palettes import Spectral10, Category10
import pandas as pd
from io import BytesIO
from PIL import Image
import base64
from bokeh.models import NumeralTickFormatter
from bokeh.models import BasicTickFormatter

output_notebook()

Loading BokehJS ...

In [16]:
p = figure(width=600, height=300, x_axis_type="log") 

# Clean top-1 accuracy
p.line(df.x, df.clean_acc1_values, line_width=2, color='#1f77b4', legend_label='Clean accuracy')
p.circle(df.x, df.clean_acc1_values, fill_color="white", size=4)

# Adversarial top-1 accuracy
p.line(df.x, df.adv_acc1_values, line_width=2, color='#ff7f0e', legend_label='Adversarial accuracy')
p.circle(df.x, df.adv_acc1_values, color='#ff7f0e', fill_color="white", size=4)

# Axes
p.xaxis.axis_label = "ϵ"
p.xaxis.axis_label_text_font_style = "normal"
p.xaxis.ticker = list(df.x)
p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
#p.xaxis.formatter.use_scientific = False

p.yaxis.axis_label = "Top-1 Accuracy"
p.yaxis.axis_label_text_font_style = "italic"


show(p);